In [1]:
import pandas as pd
import requests
from keys import KEY_TWO


In [2]:
themes = requests.get(f'https://brickset.com/api/v3.asmx/getThemes?apiKey={KEY_TWO}') 

In [3]:
data = themes.json()
df = pd.json_normalize(data, 'themes')
df.shape

(153, 5)

In [4]:
df.sort_values('theme', axis=0, ascending=True, inplace=True)
df.shape

(153, 5)

In [5]:
# mask = df[df['yearFrom'] < 1999].index
# mask_two = df[df['setCount'] < 50].index 
# mask_three = df[df['yearTo'] < 2022].index
# df.drop(mask, inplace=True)
# df.drop(mask_two, inplace=True)
# df.drop(mask_three, inplace=True)

mask = df[(df['yearFrom'] < 1999) | (df['setCount'] < 50) | (df['yearTo'] < 2022)].index
df.drop(mask, inplace=True)

In [6]:
df.shape

(19, 5)

In [7]:
mask_two = df[(df['theme'] == 'Collectable Minifigures') | (df['theme'] == 'Miscellaneous')].index
df.drop(mask_two, inplace=True)
df.shape


(17, 5)

In [8]:
theme_list = []
for item in df['theme'].sample(5):
    theme_list.append(item)
print(theme_list)

['Architecture', 'Star Wars', 'Jurassic World', 'Dots', 'Friends']


In [9]:
df.to_csv('./theme_list.csv')

In [10]:
# set_list = requests.get(f'https://brickset.com/api/v3.asmx/getYears?apiKey={KEY_TWO}&theme={theme_list}')
parameters = {'theme' : 'Star Wars'}
response = requests.get(f"https://brickset.com/api/v3.asmx/getSets?apiKey={KEY_TWO}&params={parameters}")

response
# set_data = set_list.json()
# set_data
# set_df = pd.json_normalize(set_data, 'sets')
# set_df.to_csv('./set_list.csv')

# response = requests.get(f'https://brickset.com/api/v3.asmx/getYears?apiKey={KEY_TWO}&theme={parameters}')
# response


<Response [500]>